In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

# Cargar el archivo CSV desde la ruta local con una codificación específica
df_tacos = pd.read_csv("basededatos/taco_taco_barato.csv", encoding='ISO-8859-1')

# Asegúrate de que las columnas de coordenadas existan y conviértalas en geometría
df_tacos['geometry'] = df_tacos.apply(lambda row: Point(row['Longitud'], row['Latitud']), axis=1)
gdf = gpd.GeoDataFrame(df_tacos, geometry='geometry')

# Establecer el CRS
gdf.set_crs(epsg=4326, inplace=True)

# Agregar las nuevas columnas
gdf['href'] = None  # Asigna None si no tienes columna 'url'
gdf['service_level'] = gdf['Descripcion estrato personal ocupado'].str.split(', ').apply(lambda x: len([v for v in x if "all" in v]))
gdf['lines_served'] = gdf['Nombre de la Unidad Económica'].str.split('-').apply(lambda x: len(x))

# Verifica las columnas del GeoDataFrame
print(gdf.columns)

In [ ]:
# Definir los colores para los niveles de servicio
colors = {
    0: 'blue',    
    1: 'green',
    2: 'yellow',
    3: 'red',
}

In [ ]:
# Crear un mapa centrado en una ubicación específica (ajusta las coordenadas según sea necesario)
m = folium.Map(location=[24.8, -107.39], zoom_start=13)  # Cambia la ubicación al centro de tus datos

# Añadir el GeoJson al mapa
folium.GeoJson(
    gdf,
    name="Taquerías",
    tooltip=folium.GeoJsonTooltip(fields=["Nombre de la Unidad Económica", "Nombre de clase de la actividad"]),
    popup=folium.GeoJsonPopup(fields=["Nombre de la Unidad Económica", "href", "Nombre de clase de la actividad"]),
    style_function=lambda x: {
        "fillColor": colors.get(x['properties'].get('service_level', -1), 'gray'),  
        "weight": 1,
        "fillOpacity": 0.6,
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
).add_to(m)

# Mostrar el mapa
m

In [ ]:

# Función para determinar el color y tamaño del círculo según el número de empleados
def get_circle_properties(desc):
    if "0 a 5" in desc:
        return 'green', 10  # Color y tamaño para 0-5 empleados
    elif "6 a 10" in desc:
        return 'yellow', 15  # Color y tamaño para 6-10 empleados
    elif "11 a 20" in desc:  # Ajusta según tus categorías
        return 'orange', 20  # Color y tamaño para 11-20 empleados
    else:
        return 'red', 25  # Color y tamaño para más de 20 empleados

# Añadir círculos al mapa
for _, row in gdf.iterrows():
    color, radius = get_circle_properties(row['Descripcion estrato personal ocupado'])
    
    folium.CircleMarker(
        location=[row['Latitud'], row['Longitud']],
        radius=radius,  # Tamaño del círculo basado en el número de empleados
        popup=folium.Popup(f"{row['Nombre de la Unidad Económica']}: {row['Descripcion estrato personal ocupado']}", parse_html=True),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6
    ).add_to(m)

# Mostrar el mapa
m